# Initialize Variables

In [0]:
DatasetURL = "https://uhesse.files.wordpress.com/2020/08/"
file = "mytpch_oracle.zip"
contentName = "TPC-H-SF1"
basePath = f"/user/{contentName}"
rawPath = f"{basePath}/raw"
silverPath = f"{basePath}/silver"
goldPath = f"{basePath}/gold"
driverFolder = "/databricks/driver/"
DriverPath = f"file:{driverFolder}{file}"
BaseURL = DatasetURL + file

# Download File

In [0]:
from urllib.request import urlretrieve
urlretrieve(BaseURL,file)

Out[2]: ('mytpch_oracle.zip', <http.client.HTTPMessage at 0x7fb615080ee0>)

# Unzip File

In [0]:
import zipfile
with zipfile.ZipFile(f"{driverFolder}{file}", 'r') as zip_ref:
    zip_ref.extractall(f"{driverFolder}{contentName}")

# Move txt files to Storage

In [0]:
downloadedList = [f.name for f in dbutils.fs.ls(f"file:{driverFolder}{contentName}") if f.name.endswith("txt") and f.name != "README.txt"]
for file in downloadedList:
    fileFolder = file[:-4]
    dbutils.fs.mv(f"file:{driverFolder}{contentName}/{file}", f"dbfs:{rawPath}/{fileFolder}/{file}")

# Set Schema of all tables

In [0]:
tcph_schemas = {'nation':'n_nationkey int, n_name string, n_regionkey int, n_comment string',
               'region':'r_regionkey int, r_name string, r_comment string',
               'part':'p_partkey int, p_name string, p_mfgr string, p_brand string, p_type string, p_size int, p_container string, p_retailprice decimal(15,2), p_comment string',
               'supplier':'s_suppkey int, s_name string, s_address string, s_nationkey int, s_phone string, s_acctbal decimal(4,2), s_comment string',
               'customer':'c_custkey int,c_name string, c_address string, c_nationkey int,c_phone string, c_acctbal decimal(4,2), c_mktsegment string, c_comment string',
               'partsupp':'ps_partkey int, ps_suppkey int, ps_availqty int, ps_supplycost decimal(15,2), ps_comment string',
               'orders':'o_orderkey int, o_custkey int, o_orderstatus string, o_totalprice decimal(15,2), o_orderdate date, o_orderpriority string, o_clerk string, o_shippriority int, o_comment string',
               'lineitem':'l_orderkey int, l_partkey int, l_suppkey int, l_linenumber int, l_quantity int, l_extendedprice decimal(15,2), l_discount decimal(2,2), l_tax decimal(2,2), l_returnflag string, l_linestatus string, l_shipdate date, l_commitdate date, l_receiptdate date, l_shipinstruct string, l_shipmode string, l_comment string '}

# Read the files as csv and write as Delta

In [0]:
from pyspark.sql.functions import col, trim
for name, schema in tcph_schemas.items():
    readPath = f"{rawPath}/{name}"
    writePath = f"{silverPath}/{name}"
    df = (spark
           .read
           .format('csv')
           .schema(schema)
           .load(readPath))
    string_cols = [c for c, t in df.dtypes if t =='string']
    for colname in string_cols :
        df= df.withColumn(colname, trim(col(colname)))
    (df
    .write
    .format('delta')
    .mode("overwrite")
    .option("overwriteSchema","true")
    .save(writePath))
    spark.sql(f"CREATE TABLE IF NOT EXISTS {name} USING delta LOCATION '{writePath}'")

# Setup Star Schema sql

In [0]:
from collections import OrderedDict

tpch_star_schema_sql = OrderedDict()
tpch_star_schema_sql['dimcustomer'] ="""
SELECT cus.c_custkey                AS custkey,
       cus.c_name                   AS customer_name,
       cus.c_address                AS customer_address,
       Substring(cus.c_phone, 1, 2) AS customer_country_code,
       cus.c_phone                  AS customer_phone_no,
       cus.c_acctbal                AS customer_account_balance,
       cus.c_mktsegment             AS market_segment,
       nat.n_name                   AS customer_nation,
       reg.r_name                   AS customer_region,
       cus.c_comment                AS customer_comment
FROM   customer cus
       INNER JOIN nation nat
               ON ( nat.n_nationkey = cus.c_nationkey )
       INNER JOIN region reg
               ON ( reg.r_regionkey = nat.n_regionkey ) 
"""
tpch_star_schema_sql['dimsupplier'] = """
SELECT s_suppkey  AS suppkey,
       s_name     AS supplier_name,
       s_address  AS supplier_address,
       s_phone    AS supplier_phone_no,
       s_acctbal  AS supplier_account_balance,
       s_comment  AS supplier_comment,
       nat.n_name AS supplier_nation,
       reg.r_name AS supplier_region
FROM   supplier sup
       INNER JOIN nation nat
               ON ( nat.n_nationkey = sup.s_nationkey )
       INNER JOIN region reg
               ON ( reg.r_regionkey = nat.n_regionkey ) 
"""
tpch_star_schema_sql['dimpart'] = """
SELECT p_partkey   AS partkey,
       p_name      AS part_name,
       p_mfgr      AS manufacturer,
       p_brand     AS brand,
       p_type      AS type,
       p_size      AS size,
       p_container AS container
FROM   part
"""
tpch_star_schema_sql['dimorderinfo'] = """
SELECT Row_number() OVER( ORDER BY NULL ) AS orderinfokey,
       t.*
FROM   (SELECT DISTINCT ord.o_shippriority  AS ship_priority,
                        lit.l_shipmode      AS ship_mode,
                        ord.o_orderpriority AS order_priority,
                        ord.o_orderstatus   AS order_status,
                        lit.l_shipinstruct  AS ship_instruct,
                        lit.l_returnflag    AS return_flag,
                        lit.l_linestatus    AS line_status
        FROM   orders ord
               INNER JOIN lineitem lit
                            ON ( lit.l_orderkey = ord.o_orderkey )) t 
"""
tpch_star_schema_sql['factpartsupp'] = """
SELECT ps_partkey    AS partkey,
       ps_suppkey    AS suppkey,
       ps_availqty   AS availqty,
       ps_supplycost AS supplycost
FROM   partsupp 
"""
tpch_star_schema_sql['factorderline'] = """
SELECT lit.l_partkey     AS partkey,
       lit.l_suppkey     AS suppkey,
       ord.o_custkey     AS custkey,
       ord.o_orderkey    AS orderkey,
       inf.orderinfokey  AS orderinfokey,
       lit.l_shipdate    AS shipdate,
       ord.o_orderdate   AS orderdate,
       lit.l_commitdate  AS commitdate,
       lit.l_receiptdate AS receiptdate,
       lit.l_quantity    AS quantity,
       prt.p_retailprice AS retailprice,
       lit.l_discount    AS discount,
       psp.ps_supplycost AS supplycost,
       lit.l_tax         AS tax,
       ord.o_comment     AS order_comment
FROM   orders ord
       INNER JOIN lineitem lit
                    ON ( ord.o_orderkey = lit.l_orderkey )
       INNER JOIN partsupp psp
                    ON ( psp.ps_partkey = lit.l_partkey
                         AND psp.ps_suppkey = lit.l_suppkey )
       INNER JOIN part prt
                    ON ( prt.p_partkey = lit.l_partkey )
       INNER JOIN dimorderinfo inf
                    ON ( inf.ship_priority = ord.o_shippriority
                         AND inf.ship_mode = lit.l_shipmode
                         AND inf.order_priority = ord.o_orderpriority
                         AND inf.order_status = ord.o_orderstatus
                         AND inf.ship_instruct = lit.l_shipinstruct
                         AND inf.return_flag = lit.l_returnflag
                         AND inf.line_status = lit.l_linestatus ) 
"""

# Build Star Schema "gold" tables

In [0]:
for tbl, sql in tpch_star_schema_sql.items():
    spark.sql(f'''
    CREATE OR REPLACE TABLE {tbl}
    USING DELTA 
    LOCATION '{goldPath}/{tbl}' AS
    {sql}
    ''')